In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os
import sys
import time
from hdfs import InsecureClient
import random
import json


In [2]:
client = InsecureClient('http://localhost:9870', user='big') #나의 로컬호스트주소 #user : big(내가정함)

# client.set_permission("/news_data/article", 777)


In [2]:
#!pip install hdfs

In [3]:
def execute_rest_api(method, url, headers, params):    
    if method == 'get':
        res = requests.get(url, params=params, headers=headers)
    elif method == 'post':
        res = requests.post(url, params=params, headers=headers)
    
    if res == None or res.status_code != 200:
        raise Exception('응답코드 : ' + str(res.status_code))

    return res.text

In [4]:
import datetime as dt
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [5]:
from bs4 import BeautifulSoup

file_dir = '/news_data/article/'
file_name = 'realtimenews_' + cal_std_day(0) + '.json'

cols=['TITLE']
data=[]

for page in range (1,20):
    time.sleep(random.uniform(3, 4))
    url='https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&page='+str(page)
        # url = 'https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&date=20220927&page='+str(page)
    headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
    response = requests.get(url, headers=headers)
    response_txt = response.text
    soup = BeautifulSoup(response_txt, 'html.parser')
    
    article_list=soup.select('#contentarea_left > ul > li > dl')
    
    for articles in article_list:
        for idx, article in enumerate(articles.select('a')):
            rows=[]

            news_title = article.text
         
            if len(news_title) > 0:
                rows.append(news_title)
                tmp=dict(zip(cols,rows))
        
                data.append(tmp)
                                      

res = {
    'meta':{
        'desc':'네이버 실시간 속보 크롤링',
        'cols':{
            'TITLE':'뉴스제목'
        },
        'std_day':cal_std_day(0)
    },
   'data':data
}

client.write(file_dir+file_name, json.dumps(res, ensure_ascii=False), encoding='utf-8')


In [12]:
rows

['"회계 감리 1년으로 명문화"…금감원, 외부감사 시행세칙 개정']

In [28]:
tmp

{'TITLE': "이베스트證, '2022년 하반기 대졸 채용전제형 인턴' 채용 진행"}

In [33]:
data

[{'TITLE': '[시황종합] 코스피, 미약한 반등 2170대 마감…환율, 1원 하락'},
 {'TITLE': '미래에셋증권, 중소기업퇴직연금기금제도 전담운용기관 선정'},
 {'TITLE': '코스피 소폭 상승, 삼성바이오로직스 6%↑ [마감시황]'},
 {'TITLE': '연락처·주소 누락...가상화폐거래소, 고객 관리 곳곳에 구멍'},
 {'TITLE': '[외환마감]달러 강세 확대+결제 수요…환율, 1원 하락에 그쳐'},
 {'TITLE': '닛케이 0.95% 상승했는데 코스피는 0.08% 상승에 그쳐'},
 {'TITLE': '티쓰리엔터테인먼트, 증권신고서 제출…11월 상장'},
 {'TITLE': "권순우 '혼신의 경기'"},
 {'TITLE': "권순우 '혼신의 경기'"},
 {'TITLE': "권순우 '혼신의 경기'"},
 {'TITLE': "권순우 '혼신의 경기'"},
 {'TITLE': '권순우, 코리아오픈 단식 8강 진출 실패'},
 {'TITLE': "권순우 '혼신의 경기'"},
 {'TITLE': "권순우 '혼신의 경기'"},
 {'TITLE': '공을 ?는 매서운 눈'},
 {'TITLE': '젠슨 브룩스비, 코리아오픈 단식 8강 진출'},
 {'TITLE': "권순우 '혼신의 경기'"},
 {'TITLE': '젠슨 브룩스비, 코리아오픈 단식 8강 진출'},
 {'TITLE': "권순우 '혼신의 경기'"},
 {'TITLE': '목 축이는 권순우'},
 {'TITLE': '코스피, 유럽발 경기침체 우려에 상승폭 반납…2170선 마감'},
 {'TITLE': '환율 롤러코스터 장세…1440원 아래 간신히 턱걸이'},
 {'TITLE': '[시황] 코스피, ‘외인’ 반발매수 유입에 강보합…2170.93P 마감'},
 {'TITLE': '[마감시황]코스피, 상승분 반납 후 강보합 마감…0.08%↑'},
 {'TITLE': '장 중 기업공시 [9월 29일]'},
 {'TITLE': '웹툰?맥주?게임까지…MZ세대 취향 저격 증권사'},
 {

In [32]:
len(data)

80

In [16]:
tmp

{'TITLE': '텔코웨어, SK텔레콤과 32억 규모 용역제공 계약'}

In [20]:
len(data)

139

In [8]:
rows

['톱텍, 572억 규모 SK온 헝가리공장 스마트팩토리 구축 계약']

In [61]:
len(title_list)

100

In [56]:
# !pip install BeautifulSoup4
from bs4 import BeautifulSoup

file_dir = '/news_data/article'
file_name = 'realtimenews_' + cal_std_day(0) + '.json'

cols=['STD_DAY','TITLE','URL']
data=[]
rows=[]
for page in range (1,6):
    url='https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&page='+str(page)
    # url = 'https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&date=20220927&page='+str(page)
    headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
    response = requests.get(url, headers=headers)
    response_txt=execute_rest_api('get',url,{},{})
    soup = BeautifulSoup(response_txt, 'html.parser')
    articles=soup.select('#contentarea_left > ul > li > dl > dd ')
#     title=soup.select('#contentarea_left > ul > li > dl > dd > a ')
    
#     document.querySelector("#contentarea_left > ul > li.newsList.top > dl > dd:nth-child(3) > span.wdate")
    
    for idx,atag in enumerate(articles):
        if idx ==0:
            continue
        select_atags=atag.select('a')
        print(select_atags)
        rows.append(select_atags)
        print(rows)
    print(len(rows))
#         rows=[]
#         rows.append(select_atags[0].text)
#         atags_text.append(select_atags.text)

#         spans=wdate.select('span.wdate') #안되면 그냥 span 이나 text로 해보기
        
#         rows=[]
#         rows.append(atags.text)
# #         for titles in articles:
# #             titles_text=atags.text
# #             titles_date=spans.text
# #             titles_href=atags.['href']
# #         rows.append(titles_text)        
# #             rows.append(titles_text)
        
#     for titles in title:
#         titles_text=titles.text
#         titles_href=titles['href']
#         # print(titles.text)
#         # print(titles['href'])
#         # print(press_date.text)
                    
#         title_list.append(titles_text)

[]
[[]]
[<a href="/news/news_read.naver?article_id=0003453386&amp;office_id=020&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”">외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”</a>]
[[], [<a href="/news/news_read.naver?article_id=0003453386&amp;office_id=020&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”">외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”</a>]]
[]
[[], [<a href="/news/news_read.naver?article_id=0003453386&amp;office_id=020&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”">외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”</a>], []]
[<a href="/news/news_read.naver?article_id=0005328465&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title='"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할

[]
[[], [<a href="/news/news_read.naver?article_id=0003453386&amp;office_id=020&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”">외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”</a>], [], [<a href="/news/news_read.naver?article_id=0005328465&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title='"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"'>"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"</a>], [], [<a href="/news/news_read.naver?article_id=0005328463&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)">새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)</a>], [], [<a href="/news/news_read.naver?article_id=0000562709&amp;office_id=422&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="환율 1,440원도 돌파…코스피는 2,200 붕괴">환율 1,440원도 돌파…코스피는 2,200 붕괴</a>]

[]
[[], [<a href="/news/news_read.naver?article_id=0003453386&amp;office_id=020&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”">외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”</a>], [], [<a href="/news/news_read.naver?article_id=0005328465&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title='"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"'>"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"</a>], [], [<a href="/news/news_read.naver?article_id=0005328463&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)">새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)</a>], [], [<a href="/news/news_read.naver?article_id=0000562709&amp;office_id=422&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="환율 1,440원도 돌파…코스피는 2,200 붕괴">환율 1,440원도 돌파…코스피는 2,200 붕괴</a>]

[]
[[], [<a href="/news/news_read.naver?article_id=0003453386&amp;office_id=020&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”">외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”</a>], [], [<a href="/news/news_read.naver?article_id=0005328465&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title='"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"'>"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"</a>], [], [<a href="/news/news_read.naver?article_id=0005328463&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)">새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)</a>], [], [<a href="/news/news_read.naver?article_id=0000562709&amp;office_id=422&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="환율 1,440원도 돌파…코스피는 2,200 붕괴">환율 1,440원도 돌파…코스피는 2,200 붕괴</a>]

[[], [<a href="/news/news_read.naver?article_id=0003453386&amp;office_id=020&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”">외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”</a>], [], [<a href="/news/news_read.naver?article_id=0005328465&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title='"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"'>"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"</a>], [], [<a href="/news/news_read.naver?article_id=0005328463&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)">새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)</a>], [], [<a href="/news/news_read.naver?article_id=0000562709&amp;office_id=422&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="환율 1,440원도 돌파…코스피는 2,200 붕괴">환율 1,440원도 돌파…코스피는 2,200 붕괴</a>], [

[]
[[], [<a href="/news/news_read.naver?article_id=0003453386&amp;office_id=020&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”">외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”</a>], [], [<a href="/news/news_read.naver?article_id=0005328465&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title='"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"'>"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"</a>], [], [<a href="/news/news_read.naver?article_id=0005328463&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)">새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)</a>], [], [<a href="/news/news_read.naver?article_id=0000562709&amp;office_id=422&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="환율 1,440원도 돌파…코스피는 2,200 붕괴">환율 1,440원도 돌파…코스피는 2,200 붕괴</a>]

[[], [<a href="/news/news_read.naver?article_id=0003453386&amp;office_id=020&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”">외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”</a>], [], [<a href="/news/news_read.naver?article_id=0005328465&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title='"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"'>"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"</a>], [], [<a href="/news/news_read.naver?article_id=0005328463&amp;office_id=018&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)">새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)</a>], [], [<a href="/news/news_read.naver?article_id=0000562709&amp;office_id=422&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20220928&amp;page=1" title="환율 1,440원도 돌파…코스피는 2,200 붕괴">환율 1,440원도 돌파…코스피는 2,200 붕괴</a>], [

In [ ]:
        
print(title_list)
##############################################################
url = 'https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000'
response_txt = execute_rest_api('get',url,{},{})
soup = BeautifulSoup(response_txt, 'html.parser')
trs = soup.select('#content > div.data_table.tbl_scrl_t > table > tbody > tr')

cols = ['loc', 'v1', 'v2', 'v3', 'v4']
data = []
for idx, tr in enumerate(trs):
    if idx == 0:
        continue
    th = tr.select('th')
    tds = tr.select('td')
    
    rows = []
    rows.append(th[0].text.replace(' ', '').replace('\r\n', ''))
    
    for idx, td in enumerate(tds):
        if idx % 2 == 0:
            continue            
        rows.append(td.text.replace(' ', '').replace('\r\n', '').replace(',' , ''))
        
        tmp = dict(zip(cols, rows))
    data.append(tmp)

res = {
    'meta':{
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
            'loc':'지역'
            ,'v1':'1차접종자수'
            ,'v2':'2차접종자수'
            ,'v3':'3차접종자수'
            ,'v4':'4차접종자수'
        },
        'std_day':cal_std_day(1)
    },
   'data':data
}

client.write(file_dir+file_name, json.dumps(res, ensure_ascii=False), encoding='utf-8')

In [57]:
title_list =[]
for page in range (1,6):
    url='https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&page='+str(page)
    # url = 'https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&date=20220927&page='+str(page)
   
    headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title=soup.select('#contentarea_left > ul > li > dl > dd > a ')
    # press_date =soup.select('#contentarea_left > ul > li > dl > dd) > span.wdate')
    # title=soup.select('#contentarea_left > ul > li.newsList.top > dl > dd > a ')
    
    for titles in title:
        titles_text=titles.text
        titles_href=titles['href']
        # print(titles.text)
        # print(titles['href'])
        # print(press_date.text)
                    
        title_list.append(titles_text)
        
print(title_list)

['국채 폭락 수습 나선 영국·치매 치료 기대에 폭등한 바이오젠 [신인규의 글로벌마켓 A/S]', '외국인-기관 쌍끌이 매도, 코스피 2200 붕괴…금융위 “증안펀드 재가동”', '"영란은행 역주행, 출구전략 더 꼬인다…연준도 비슷한 상황 처할 수도"', '새 英정부가 망쳐놓은 금융시장, 영란은행이 소방수로(종합)', '환율 1,440원도 돌파…코스피는 2,200 붕괴', "전세계 증시 불안한데…연준은 마이웨이, '페드 풋' 없다[오미주]", '한국투자증권, 글로벌 IB 美 스티펄과 합작회사 설립한다', '[5분세계증시] 코스피 2,200 붕괴…코스피 2.45%·코스닥 3.47% 폭락', "[시그널] 사학연금 4000억 출자…'돈 가뭄' M&A시장에 단비", "SM 주식 판 강타, 스톡옵션 83% '대박'…보아도 '억 소리'", '"광고형 요금제, 3년내 전체매출 최대 20%…넷플릭스 주식 사라"', '英중앙은행, 금융시장 안정 위해 긴급 국채매입…양적긴축 연기', '증시 또 발작… 2200마저 무너져', '초일류·초격차 기술 확보한다…"AI 경쟁력 세계 3위로"', '65년래 최악의 금리상승…국채 팔려던 영란銀, 다시 국채 산다', '반대매매도 증가세…늘어나는 증시 이탈', '서학개미 베팅한 엔비디아...월가도 긍정적 평가 [GO WEST]', '금융위 "증권시장 안정펀드 재가동 준비"', '테슬라에 `매도`의견 고수한 씨티 "車생산량·유럽경기 우려"', '[마켓인]환율폭주에 또 국민연금 주목…해외투자 비중 조절하나', '"그래도 너 밖에 없다"...꾸준히 돈 몰리는 00펀드', '증시 덮친 \'검은 수요일\'…"환율, 문제는 속도야" [증시프리즘]', 'MBN 뉴스7 앵커 오프닝 - 9월 28일', "[뉴스추적] 킹달러 폭주에 검은 9월…'퍼펙트 스톰' 오나?", '환율 1,440원 찍었는데 1,298원에 환전…토스증권 해프닝(종합2보)', '[데이터로 보는 증시]유형별 펀드 자금 동향( 9월 27일)', '코스피 2,200도 무너졌다…환율은 장중 1,44

In [ ]:
from bs4 import BeautifulSoup

file_dir = '//'
file_name = 'financenews_' + cal_std_day(0) + '.json'


title_list =[]
for page in range (1,6):
    url='https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&page='+str(page)
    # url = 'https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&date=20220927&page='+str(page)
   
    headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title=soup.select('#contentarea_left > ul > li > dl > dd > a ')
    # press_date =soup.select('#contentarea_left > ul > li > dl > dd) > span.wdate')
    # title=soup.select('#contentarea_left > ul > li.newsList.top > dl > dd > a ')
    

    
    for titles in title:
        titles_text=titles.text
        titles_href=titles['href']
        # print(titles.text)
        # print(titles['href'])
        # print(press_date.text)
        
            
        title_list.append(titles_text)
        

print(title_list)


In [59]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+str(month)+str(day)

In [63]:
# result_list =[]
error_cnt =0 

def naver_news_title(dates):
    base_url = 'https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&date={}'
    headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
    
    for date in dates:
        url=base_url+cal_std_day(0)
        response = requests.get(url, headers=headers)

        if response.status_code ==200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            title_list =[]
# title=soup.select('#contentarea_left > ul > li.newsList.top > dl > dd > a ')
            for title in title_list:
                try :
                    title=soup.select('#contentarea_left > ul > li > dl > dd > a ')
                    print(titles.text)
                    print(titles['href'])
                except:
                    error_cnt +=1

                    
pd.DataFrame(title_list)
	

""


In [58]:
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&date=20220927'
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
response = requests.get(url, headers=headers)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
title=soup.select('#contentarea_left > ul > li > dl > dd > a ')
# title=soup.select('#contentarea_left > ul > li.newsList.top > dl > dd > a ')
for titles in title:
    print(titles.text)
    print(titles['href'])

len(title)
    #contentarea_left > ul > li.newsList.top > dl > dd:nth-child(2) > a

유상호 한투증권 부회장, 차기 금투협회장 선거 안나간다
/news/news_read.naver?article_id=0000174994&office_id=629&mode=LSS2D&type=0§ion_id=101§ion_id2=258§ion_id3=&date=20220927&page=1
리튬플러스, 조달청서 리튬 원재료 대여 승인 받아
/news/news_read.naver?article_id=0005327379&office_id=018&mode=LSS2D&type=0§ion_id=101§ion_id2=258§ion_id3=&date=20220927&page=1
中 니오 전기차 배터리용 리튬광산 투자, 스타트업 최초
/news/news_read.naver?article_id=0004905130&office_id=014&mode=LSS2D&type=0§ion_id=101§ion_id2=258§ion_id3=&date=20220927&page=1
"파운드화 반등에 안도"…코스피·코스닥 강보합 마감
/news/news_read.naver?article_id=0001056555&office_id=215&mode=LSS2D&type=0§ion_id=101§ion_id2=258§ion_id3=&date=20220927&page=1
금융노조 2차 총파업 다음 달로 연기... "금융소비자 불편 고려"
/news/news_read.naver?article_id=0000699098&office_id=469&mode=LSS2D&type=0§ion_id=101§ion_id2=258§ion_id3=&date=20220927&page=1
모델솔루션 일반청약 경쟁률 1천515대 1…증거금 5조원
/news/news_read.naver?article_id=0013466576&office_id=001&mode=LSS2D&type=0§ion_id=101§ion_id2=258§ion_id3=&date=20220927&page=1
[IPO출사표]탑머티리얼 "차세대 2차전지 소재 대표기업 도약"
/

15